In [1]:
import causal_nf.utils.wandb_local as wandb_local
import causal_nf.config as causal_nf_config
from causal_nf.config import cfg
import causal_nf.utils.training as causal_nf_train
import os
import glob
import argparse
from yacs.config import CfgNode 
import torch
import causal_nf.utils.io as causal_nf_io
import time
import numpy as np

/Users/llaurabat/micromamba/envs/causal-flows/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from causal_nf.preparators.FF_preparator import FFPreparator
from causal_nf.config import cfg


In [3]:
# ckpt_file = "/Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final/edf3624c525411ef8eecacde48001122/last.ckpt"
# ckpt_file = "/Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final_adj_ON/1f995ea8525c11efbcefacde48001122/last.ckpt"
# ckpt_file = "/Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final_ate1/a6cbbd08526111efa71facde48001122/last.ckpt"
ckpt_file= "/Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final_ate1_check/9d16ffae528211ef985aacde48001122/last.ckpt"


In [4]:
# args_list = []
# args =  CfgNode({'config_file': 'output_FF_final/edf3624c525411ef8eecacde48001122/wandb_local/config_local.yaml', 
#                  'config_default_file': 'output_FF_final/edf3624c525411ef8eecacde48001122/wandb_local/default_config.yaml', 
#                  'project': None, 'wandb_mode': 'disabled', 'wandb_group': None, 
#                  'load_model': 'output_FF_final/edf3624c525411ef8eecacde48001122', 'delete_ckpt': False})


In [5]:
# args_list = []
# args =  CfgNode({'config_file': 'output_FF_final_adj_ON/1f995ea8525c11efbcefacde48001122/wandb_local/config_local.yaml', 
#                  'config_default_file': 'output_FF_final_adj_ON/1f995ea8525c11efbcefacde48001122/wandb_local/default_config.yaml', 
#                  'project': None, 'wandb_mode': 'disabled', 'wandb_group': None, 
#                  'load_model': 'output_FF_final_adj_ON/1f995ea8525c11efbcefacde48001122', 'delete_ckpt': False})

In [6]:
# args_list = []
# args =  CfgNode({'config_file': 'output_FF_final_ate1/a6cbbd08526111efa71facde48001122/wandb_local/config_local.yaml', 
#                  'config_default_file': 'output_FF_final_ate1/a6cbbd08526111efa71facde48001122/wandb_local/default_config.yaml', 
#                  'project': None, 'wandb_mode': 'disabled', 'wandb_group': None, 
#                  'load_model': 'output_FF_final_ate1/a6cbbd08526111efa71facde48001122', 'delete_ckpt': False})

In [7]:
args_list = []
args =  CfgNode({'config_file': 'output_FF_final_ate1_check/9d16ffae528211ef985aacde48001122/wandb_local/config_local.yaml', 
                 'config_default_file': 'output_FF_final_ate1_check/9d16ffae528211ef985aacde48001122/wandb_local/default_config.yaml', 
                 'project': None, 'wandb_mode': 'disabled', 'wandb_group': None, 
                 'load_model': 'output_FF_final_ate1_check/9d16ffae528211ef985aacde48001122', 'delete_ckpt': False})

In [8]:
# args_list, args = causal_nf_config.parse_args()

In [9]:
config = causal_nf_config.build_config(
    config_file=args.config_file,
    args_list=args_list,
    config_default_file=args.config_default_file,
)

In [27]:
cfg.dataset

CfgNode({'root': '../Data', 'name': 'FF_data', 'sem_name': 'dummy', 'splits': [0.8, 0.1, 0.1], 'k_fold': 1, 'shuffle_train': True, 'single_split': False, 'loss': 'default', 'scale': 'default', 'num_samples': 1000, 'base_version': 1, 'add_noise': False, 'type': 'torch', 'use_edge_attr': False})

In [11]:
causal_nf_config.assert_cfg_and_config(cfg, config)

In [12]:
preparator = FFPreparator.loader(cfg.dataset)

In [13]:
preparator.prepare_data()


Preparing data...

Preparing data...

Preparing data...


In [14]:
# IMPORTANT: insert ckpt file otherwise it loads anyway bt something wrong
model_lightning = causal_nf_train.load_model(cfg=cfg, preparator=preparator, ckpt_file=ckpt_file)

NSF(
  (transforms): ModuleList(
    (0): MaskedAutoregressiveTransform(
      (base): MonotonicRQSTransform(bins=8)
      (order): [0, 1, 2, 3, 4, 5]
      (hyper): MaskedMLP(
        (0): MaskedLinear(in_features=6, out_features=32, bias=True)
        (1): ELU(alpha=1.0)
        (2): MaskedLinear(in_features=32, out_features=32, bias=True)
        (3): ELU(alpha=1.0)
        (4): MaskedLinear(in_features=32, out_features=32, bias=True)
        (5): ELU(alpha=1.0)
        (6): MaskedLinear(in_features=32, out_features=138, bias=True)
      )
    )
  )
  (base): DiagNormal(loc: torch.Size([6]), scale: torch.Size([6]))
)
[INFO] Loading <class 'causal_nf.models.causal_nf.CausalNFightning'> from /Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final_ate1_check/9d16ffae528211ef985aacde48001122/last.ckpt
scaler_transform StandardTransform(shift=tensor([2.7234, 2.7408, 2.7082, 2.6903, 0.9523, 1.9587]), scale=tensor([2.7295, 2.6963, 2.7701, 2.7185, 0.2133, 1.0559]))
S

In [15]:
# list(model.model.named_parameters())

In [16]:
model = model_lightning.model
model.eval()

CausalNormalizingFlow(
  (flow): NSF(
    (transforms): ModuleList(
      (0): MaskedAutoregressiveTransform(
        (base): MonotonicRQSTransform(bins=8)
        (order): [0, 1, 2, 3, 4, 5]
        (hyper): MaskedMLP(
          (0): MaskedLinear(in_features=6, out_features=32, bias=True)
          (1): ELU(alpha=1.0)
          (2): MaskedLinear(in_features=32, out_features=32, bias=True)
          (3): ELU(alpha=1.0)
          (4): MaskedLinear(in_features=32, out_features=32, bias=True)
          (5): ELU(alpha=1.0)
          (6): MaskedLinear(in_features=32, out_features=138, bias=True)
        )
      )
    )
    (base): DiagNormal(loc: torch.Size([6]), scale: torch.Size([6]))
  )
)

In [17]:
loaders = preparator.get_dataloaders(
    batch_size=cfg.train.batch_size, num_workers=cfg.train.num_workers
)

for i, loader in enumerate(loaders):
    causal_nf_io.print_info(f"[{i}] num_batchees: {len(loader)}")


[INFO] [0] num_batchees: 1
[INFO] [1] num_batchees: 1
[INFO] [2] num_batchees: 1


In [18]:
batch = next(iter(loaders[-1]))

In [42]:
batch[0][:,-1]

tensor([ 4.1480e+00,  2.0225e-01,  1.7911e+00, -1.0034e+00,  2.1222e+00,
         1.2361e+00,  3.3899e+00,  2.1877e+00,  2.1903e+00,  1.0463e+00,
         3.3979e+00,  2.5997e+00,  3.3265e+00,  1.9016e+00,  2.4261e-01,
         1.6047e+00,  2.1983e+00,  1.5351e+00,  9.1221e-01,  9.4868e-01,
         2.1077e+00,  2.3309e+00,  7.7667e-01,  3.8567e-01,  1.1713e+00,
         6.5297e-01,  3.6031e-01,  2.5990e+00,  2.3453e+00,  2.6452e+00,
         2.1796e+00,  2.4353e+00,  3.1512e-01,  1.1139e+00,  2.5765e+00,
         9.7517e-02,  3.6379e+00,  1.3688e+00,  2.4686e+00,  3.9479e+00,
         3.7692e-01,  3.1133e+00,  3.0773e+00,  5.0698e-01,  5.7148e-01,
         1.8828e+00,  2.1106e+00,  2.3609e+00,  3.1870e+00,  1.1142e+00,
         1.8064e+00,  2.1699e+00,  2.4277e+00,  1.9419e+00,  3.5183e+00,
         1.7785e+00,  3.9868e-01,  2.1339e+00,  9.9808e-01,  1.4953e+00,
         3.7664e+00,  3.8612e+00,  4.0687e+00,  2.1283e+00,  8.5413e-01,
         4.2193e-01,  1.8101e+00,  1.1327e+00, -6.6

In [39]:
batch[1]

tensor([ 0.7386,  1.1559,  4.3890,  1.3708,  3.2283,  1.9757,  1.4864,  0.6046,
         2.3884,  1.1410,  3.3498,  1.1096,  3.1578,  4.2387,  0.4782,  2.0617,
         1.0176,  2.1076,  2.3509,  2.7729,  1.9457,  2.2885, -0.3038,  1.0378,
         0.4936,  2.4642,  3.0313,  2.6664,  1.3960,  0.9151,  2.3526,  1.8357,
         2.1642,  2.0526,  3.2073,  1.7686,  1.6532,  3.5243,  3.3730,  1.9575,
         0.6292,  2.6592,  2.8368,  2.8270,  2.3554,  3.3087,  2.3161,  1.0955,
         4.7291,  2.2746,  0.4864,  1.5052,  1.9754,  3.3743,  1.9575,  3.6084,
         2.3448, -0.0455,  2.2315,  3.1405,  0.6969,  3.4210,  2.4723,  1.7862,
         1.0641,  0.3242,  3.1868,  1.4764,  2.7546,  2.2155,  1.0719,  3.1571,
         2.6292,  1.0948,  2.0707,  1.7225,  2.6315,  2.7319, -1.5639,  0.5955,
         3.0438,  1.1719, -0.6786,  3.1406,  0.9944,  0.9930,  1.6354,  1.8626,
         2.6885,  2.9180,  2.1727, -0.4518,  2.1431,  2.3909,  3.0199,  2.0408,
         1.2558,  3.2367,  1.9096,  1.88

In [20]:
# predict from model class
# torch.random.manual_seed(8)
# loss_dict = model_lightning.predict(
#     batch=next(iter(loaders[2])),
#     observational=False,
#     intervene=False,
#     counterfactual=False,
#     ate=True,
# )


In [45]:
output = {}
x = batch[0] # is this u or x? I hope x
n = x.shape[0]
with torch.enable_grad():
    output["log_prob_true"] = preparator.log_prob(x)

log_prob = model.log_prob(x, scaler=preparator.scaler_transform)
output["loss"] = -log_prob
output["log_prob"] = log_prob
intervention_list = preparator.get_ate_list()

int_dict = {'name': '1_0', 'a': 1., 'b': 0., 'index': 4}


name = int_dict["name"]
a = int_dict["a"]#1.
b = int_dict["b"]#0.
index = int_dict["index"]

ate = model_lightning.model.compute_ate(
    index,
    a=a,
    b=b,
    num_samples=10000,
    scaler=preparator.scaler_transform,
)
ate

tensor([-0.0087, -0.0932,  0.0234, -0.0100,  0.7340])

In [57]:
# manual implementation of my_predict
n_rounds = 5
ates = []
seeds = np.arange(n_rounds)
for i, seed in enumerate(seeds):
    output = {}
    x = batch[0] # is this u or x? I hope x
    n = x.shape[0]
    with torch.enable_grad():
        output["log_prob_true"] = preparator.log_prob(x)
    
    log_prob = model.log_prob(x, scaler=preparator.scaler_transform)
    output["loss"] = -log_prob
    output["log_prob"] = log_prob
    # intervention_list = preparator.get_ate_list()
    
    int_dict = {'name': '1_0', 'a': 1., 'b': 0., 'index': 4}
    

    name = int_dict["name"]
    a = int_dict["a"]#1.
    b = int_dict["b"]#0.
    index = int_dict["index"]

    torch.random.manual_seed(seed) 
    ate = model_lightning.model.compute_ate(
        index,
        a=a,
        b=b,
        num_samples=10000,
        scaler=preparator.scaler_transform,
    )
    
    ates.append(ate.detach().numpy())
    print(ates)

ates = np.array(ates)
ates_y = ates[:,-1]
mean = ates_y.mean()
std =  ates_y.std()
print('mean:', mean)
print('std:', std)

[array([-0.02142811,  0.01597714, -0.03495955, -0.0507586 ,  0.7280544 ],
      dtype=float32)]
[array([-0.02142811,  0.01597714, -0.03495955, -0.0507586 ,  0.7280544 ],
      dtype=float32), array([-5.2449703e-03,  1.0221481e-02,  8.2733631e-03,  7.2073936e-04,
        7.5292134e-01], dtype=float32)]
[array([-0.02142811,  0.01597714, -0.03495955, -0.0507586 ,  0.7280544 ],
      dtype=float32), array([-5.2449703e-03,  1.0221481e-02,  8.2733631e-03,  7.2073936e-04,
        7.5292134e-01], dtype=float32), array([ 0.02558947, -0.00136757, -0.01175404, -0.00143647,  0.74003637],
      dtype=float32)]
[array([-0.02142811,  0.01597714, -0.03495955, -0.0507586 ,  0.7280544 ],
      dtype=float32), array([-5.2449703e-03,  1.0221481e-02,  8.2733631e-03,  7.2073936e-04,
        7.5292134e-01], dtype=float32), array([ 0.02558947, -0.00136757, -0.01175404, -0.00143647,  0.74003637],
      dtype=float32), array([0.04610872, 0.06636262, 0.00415158, 0.06623077, 0.7587135 ],
      dtype=float32)]
[ar

In [65]:
def my_predict(
        self,
        batch,
        observational=False,
        intervene=False,
        counterfactual=False,
        ate=False,
        intervention_list=None,
    ):
        output = {}
        x = batch[0].to(self.device)
        n = x.shape[0]
        with torch.enable_grad():
            output["log_prob_true"] = self.preparator.log_prob(x)

        tic = time.time()
        log_prob = self.model.log_prob(x, scaler=self.preparator.scaler_transform)
        output["time_log_prob"] = self.compute_time(tic, n)
        output["loss"] = -log_prob
        output["log_prob"] = log_prob

        if ate:
            if intervention_list is None:
                intervention_list = self.preparator.get_ate_list()
            delta_times = []
            for int_dict in intervention_list:
                name = int_dict["name"]
                a = int_dict["a"]
                b = int_dict["b"]
                index = int_dict["index"]
                tic = time.time()
                ate = self.model.compute_ate(
                    index,
                    a=a,
                    b=b,
                    num_samples=10000,
                    scaler=self.preparator.scaler_transform,
                )
                delta_times.append(self.compute_time(tic, 10000))

                ate_true = self.preparator.compute_ate(
                    index, a=a, b=b, num_samples=10000
                )

                diff_ate = ate_true - ate

                rmse = torch.sqrt((diff_ate**2).sum())
                output[f"rmse_ate_x{index + 1}={name}"] = rmse

            delta_time = torch.stack(delta_times).mean()
            output["time_ate"] = delta_time

        return output

In [66]:
intervention_list = [{'name': '1_0', 'a': 1., 'b': 0., 'index': 0},]

In [67]:
bri = my_predict(
        self=model_lightning,
        batch=batch,
        observational=False,
        intervene=False,
        counterfactual=False,
        ate=True,
        intervention_list=intervention_list,
    )
bri

{'log_prob_true': tensor([-2.8672, -2.4891, -6.4312,  ..., -4.2171, -2.1910, -1.8383]),
 'time_log_prob': tensor(0.9432),
 'loss': tensor([2.5407, 2.3248, 7.3806,  ..., 4.2023, 2.1255, 2.3977],
        grad_fn=<NegBackward0>),
 'log_prob': tensor([-2.5407, -2.3248, -7.3806,  ..., -4.2023, -2.1255, -2.3977],
        grad_fn=<AddBackward0>),
 'rmse_ate_x1=1_0': tensor(0.0135),
 'time_ate': tensor(4.4213)}

In [ ]:
python main.py --config_file causal_nf/configs/FF_configs.yaml --wandb_mode disabled --project FF_data
